In [1]:
import numpy as np
import scipy.optimize as optimize

In [2]:
""" Optimization Algorithm """
""" New Matrix """
def newMat(x, Vt, k):
  V_new = np.zeros((Vt.shape), dtype=np.cfloat)
  if k==2:
    V_new[0] = np.cos(x[0])
    V_new[1] = (np.sin(x[0])) * np.exp(1j*x[1])
  elif k==3:
    V_new[0] = np.cos(x[0])
    V_new[1] = (np.sin(x[0])) * (np.cos(x[1])) * np.exp(1j*x[2])
    V_new[2] = (np.sin(x[0])) * (np.sin(x[1])) * np.exp(1j*x[3])
  else:
    V_new[0] = (np.cos(x[0])) * (np.cos(x[1]))
    V_new[1] = (np.cos(x[0])) * (np.sin(x[1])) * np.exp(1j*x[3])
    V_new[2] = (np.sin(x[0])) * (np.cos(x[2])) * np.exp(1j*x[4])
    V_new[3] = (np.sin(x[0])) * (np.sin(x[2])) * np.exp(1j*x[5])
  return V_new


""" Cost Function """
def costFn(x, Ut, Vt, A, k):
    V_new = newMat(x, Vt, k)
    Bp = np.dot(Ut, V_new) 
    loss = np.linalg.norm(A - Bp*np.conjugate(Bp))
    return (loss)


In [3]:
def calcResults(k):
    final_res = []
    m = k+9
    for n in range(k+1, k+9):
        print ("m = ",m,", n = ",n)
        res = np.zeros((100,3))
        for i in range(100):
            A = np.random.rand(m, n)
            A = A/A.sum(axis=0)         # Optimize column-wise

            #Classic Truncated SVD
            U, L, V = np.linalg.svd(A, full_matrices=False)
            Ut = U[:, :k]
            Vt = V[:k]
            Lt = L[:k]
            At = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
            res[i][0] = (np.linalg.norm(A - At))

            #Complex SVD with k = k_original
            B = np.sqrt(A)
            U, L, V = np.linalg.svd(B, full_matrices=False)
            Ut = U[:, :k]
            Vt = V[:k]
            Lt = L[:k]
            initial_guess = np.ones((2*n*(k-1),), dtype=np.longdouble)
            V_new = np.zeros(Vt.shape, dtype=np.cfloat)
            for col in range(Vt.shape[1]):
                result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Vt[:, col],A[:,col],k),
                                        tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
                V_new[:,col] = newMat(result.x, Vt[:, col], k)
            Bp = np.dot(Ut, V_new)  
            res[i][1] = (np.linalg.norm(A - np.conjugate(Bp)*Bp))

            # Complex SVD with k = k_original - 1
            k_new = k -1
            Ut = U[:, :k_new]
            Vt = V[:k_new]
            Lt = L[:k_new]
            initial_guess = np.ones((2*n*(k_new-1),), dtype=np.longdouble)
            V_new = np.zeros(Vt.shape, dtype=np.cfloat)
            for col in range(Vt.shape[1]):
                result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Vt[:, col],A[:,col],k_new),
                                        tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
                V_new[:,col] = newMat(result.x, Vt[:, col], k_new)
            Bp = np.dot(Ut, V_new)  
            res[i][2] = (np.linalg.norm(A - np.conjugate(Bp)*Bp))


            if i%10==0: print(i, end=' ')
        print('\n')
        final_res.append([m, n, *res.mean(axis=0)])
    return final_res

In [4]:
res = calcResults(k=3)

m =  12 , n =  4
0 10 20 30 40 50 60 70 80 90 

m =  12 , n =  5
0 10 20 30 40 50 60 70 80 90 

m =  12 , n =  6
0 10 20 30 40 50 60 70 80 90 

m =  12 , n =  7
0 10 20 30 40 50 60 70 80 90 

m =  12 , n =  8
0 10 20 30 40 50 60 70 80 90 

m =  12 , n =  9
0 10 20 30 40 50 60 70 80 90 

m =  12 , n =  10
0 10 20 30 40 50 60 70 80 90 

m =  12 , n =  11
0 10 20 30 40 50 60 70 80 90 



In [5]:
res

[[12, 4, 0.10730643614819792, 0.10132962553801118, 0.19278791055682223],
 [12, 5, 0.16268263591752166, 0.1542377655459289, 0.23366669715729546],
 [12, 6, 0.20450972741764642, 0.1925885668873047, 0.2763660492271491],
 [12, 7, 0.2416552205302714, 0.22733485657087152, 0.3149129499349308],
 [12, 8, 0.2751859029654912, 0.25799796636134575, 0.34679195708769434],
 [12, 9, 0.31008530176619087, 0.29078994912690226, 0.3800767171118419],
 [12, 10, 0.3310821908804019, 0.3091940114946016, 0.40069503116753785],
 [12, 11, 0.35662806566115357, 0.3325111435920211, 0.42656913599253504]]